In [ ]:
import cv2
import numpy as np
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import math
import glob
import json
import time
import os

# Define camera parameters

In [ ]:
# Camera intrinsic mattrix
camera_matrix = np.array([[606.90486004,   0.        , 326.68717083],
                          [  0.        , 608.62302034, 249.21760803],
                          [  0.        ,   0.        ,   1.        ]])
# Distorsion parameters
dist_coeffs = np.array([[ 1.11494374e-01,  5.93164264e-02,  3.38038952e-03,-3.76295683e-05, -1.07614091e+00]])

# Function to project object mask

In [ ]:
def ply_vtx(pth):
    f = open(pth)
    assert f.readline().strip() == "ply"
    f.readline()
    f.readline()
    f.readline()
    N = int(f.readline().split()[-1])
    while f.readline().strip() != "end_header":
        continue
    pts = []
    for _ in range(N):
        pts.append(np.float32(f.readline().split()[:3]))
    return np.array(pts)

def get_model_points():
    pointxyz = ply_vtx(r".\powerdrill.ply")
    dellist = [j for j in range(0, len(pointxyz))]
    dellist = random.sample(dellist, len(pointxyz) - 2000)
    pointxyz = np.delete(pointxyz, dellist, axis=0)
    return pointxyz

def project_p3d(p3d, cam_scale, K):
    if type(K) == str:
        K = intrinsic_matrix[K]
    p3d = p3d * cam_scale
    p2d = np.dot(p3d, K.T)
    p2d_3 = p2d[:, 2]
    p2d_3[np.where(p2d_3 < 1e-8)] = 1.0
    p2d[:, 2] = p2d_3
    p2d = np.around((p2d[:, :2] / p2d[:, 2:])).astype(np.int32)
    return p2d

def draw_p2ds(img, p2ds, r=1, color=[(255, 0, 0)]):
    if type(color) == tuple:
        color = [color]
    if len(color) != p2ds.shape[0]:
        color = [color[0] for i in range(p2ds.shape[0])]
    h, w = img.shape[0], img.shape[1]
    for pt_2d, c in zip(p2ds, color):
        pt_2d[0] = np.clip(pt_2d[0], 0, w)
        pt_2d[1] = np.clip(pt_2d[1], 0, h)
        img = cv2.circle(
            img, (pt_2d[0], pt_2d[1]), r, c, -1
        )
    return img

# First image

The first image is processed with the icP algorithm to find the pose of the object relative to the camera. 
Now the pose of the origin of the markers is caculated relative to the object. 

The first step is to find the pose of the markers relative to the camera. 
Then the origin of the markers relative to the object can be calculated.

In [ ]:
# Create the aruco dictionary and detector
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL) 
parameters = cv2.aruco.DetectorParameters_create()

rvec, tvec = 0, 0

# Load the image
images = glob.glob('path_to_first_image')


for image_file in images:
    # Load image
    frame = cv2.imread(image_file)

    # Detect the aruco markers in the frame
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    # If any markers are detected
    if ids is not None:
        # Estimate the pose of each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, dist_coeffs)

        # Draw the axes of each marker
        #for i in range(len(ids)):
            #cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.1)

        # Draw the IDs of each marker
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        # Estimate the pose of all the markers together
        aruco_board = cv2.aruco.GridBoard.create(6, 9, 0.05, 0.01, aruco_dict, 0) 
        _, rvec, tvec = cv2.aruco.estimatePoseBoard(corners, ids, aruco_board, camera_matrix, dist_coeffs, rvec, tvec)
        
        # Make transformation matrix
        first_pose = np.eye(4)
        first_pose[:3,:3] = cv2.Rodrigues(rvec)[0] 
        first_pose[:3,3] = tvec[:,0] 

        print(first_pose)
        cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.3)
              
    # Display the frame
    cv2.imshow('frame', frame)
    # Wait until a key is pressed
    cv2.waitKey(0)

# Close the windows
cv2.destroyAllWindows()

In [ ]:
# transformation (or pose) of hte object relative to the camera
pose_object = np.array(  [[-0.96273393,  0.03732234,  0.2678627 , -0.03810231],
                          [ 0.22752957,  0.64717135,  0.72759848, -0.04567095],
                          [-0.14619739,  0.76143042, -0.63154575,  0.76677086],
                          [ 0.        ,  0.        ,  0.        ,  1.        ]])

# transformation of the markers relative to the object is calculated 
pose_markers = np.linalg.inv(first_pose) @ pose_object

print("pose object to markers:", pose_markers)

print("pose object to camera:", first_pose @ pose_markers)

In [ ]:
# vizualize the first pose
mesh_pts = get_model_points().copy()
pose_object = first_pose @ pose_markers
mesh_pts = np.dot(mesh_pts, pose_object[:3, :3].T) + pose_object[:3, 3]

mesh_p2ds = project_p3d(mesh_pts, 1.0, camera_matrix)

np_rgb = cv2.imread('./RealSense_images/Pose_2/color/000046.png')
color = (255,0,0)
np_rgb = draw_p2ds(np_rgb, mesh_p2ds, color=color)

plt.figure()
plt.imshow(np_rgb)
plt.show()

# Process

The transformation of the markers relative to the object is calculed. Now the rest of the images are processed. 

In [ ]:
# Create the aruco dictionary and detector
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL) 
parameters = cv2.aruco.DetectorParameters_create()

rvec, tvec = 0, 0
# Load the image
images = glob.glob('path_to_images')

# Loop over all calibration images
for image_file in images:
    # Load image
    frame = cv2.imread(image_file)

    # Detect the aruco markers in the frame
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    # If any markers are detected
    if ids is not None:
        # Estimate the pose of each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, dist_coeffs)

        # Draw the axes of each marker
        #for i in range(len(ids)):
            #cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.1)

        # Draw the IDs of each marker
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        # Estimate the pose of all the markers together
        aruco_board = cv2.aruco.GridBoard.create(6, 9, 0.05, 0.01, aruco_dict, 0) 
        _, rvec, tvec = cv2.aruco.estimatePoseBoard(corners, ids, aruco_board, camera_matrix, dist_coeffs, rvec, tvec)
        
        markers_pose = np.eye(4)
        markers_pose[:3,:3] = cv2.Rodrigues(rvec)[0] 
        markers_pose[:3,3] = tvec[:,0] 
        
        # Draw origin of markers
        cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.3)
        
        # Make transformation matrix
        pose_object_to_markers = np.eye(4)
        pose_object_to_markers[:3,:3] = cv2.Rodrigues(rvec)[0] 
        pose_object_to_markers[:3,3] = tvec[:,0] 
        
        # Total pose
        total_pose =  pose_object_to_markers @ pose_markers
        
        # Project mask of 3D object model
        mesh_pts = get_model_points().copy()
        mesh_pts = np.dot(mesh_pts, total_pose[:3, :3].T) + total_pose[:3, 3]

        mesh_p2ds = project_p3d(mesh_pts, 1.0, camera_matrix)
        color = (0,0,255)
        frame = draw_p2ds(frame, mesh_p2ds, color=color)
              
    # Display the frame
    cv2.imshow('frame', frame)
    
    keys = cv2.waitKey(0)
    # Exit if the user presses 'q'
    if keys == 'q':
        break

# Close the windows
cv2.destroyAllWindows()

For saving the annotations, run the following code. 

In [ ]:
def save_transformation( trans, name):
    f_out = open( name, 'w')
    for i in range(len(trans)):
        transform = {"transformation4x4": [trans[i].tolist()]}
        json.dump( transform, f_out)
        f_out.write('\n')

In [ ]:
# Create the aruco dictionary and detector
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL) 
parameters = cv2.aruco.DetectorParameters_create()

rvec, tvec = 0, 0
# Load the image
images = glob.glob('path_to_images')

# Loop over all calibration images
for image_file in images:
    # Load image
    frame = cv2.imread(image_file)

    # Detect the aruco markers in the frame
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    # If any markers are detected
    if ids is not None:
        # Estimate the pose of each marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, dist_coeffs)

        # Estimate the pose of all the markers together
        aruco_board = cv2.aruco.GridBoard.create(6, 9, 0.05, 0.01, aruco_dict, 0) 
        _, rvec, tvec = cv2.aruco.estimatePoseBoard(corners, ids, aruco_board, camera_matrix, dist_coeffs, rvec, tvec)
        
        # Make transformation matrix      
        pose_object_to_markers = np.eye(4)
        pose_object_to_markers[:3,:3] = cv2.Rodrigues(rvec)[0] 
        pose_object_to_markers[:3,3] = tvec[:,0] 
        
        # Total pose
        total_pose =  pose_object_to_markers @ pose_markers
              
    list_poses.append(total_pose)

save_transformation(list_poses, "transs.json")